# 1. Projeto, Data Understanding & Setup

#### 1.1 - Definição do Projeto & Motivação

> No dia 05/05/2023 a Organização Mundial da Saúde (OMS) declarou o fim da Emergência de Saúde Pública de Importância Internacional referente à COVID-19. [Fonte :OMS](https://www.paho.org/pt/noticias/5-5-2023-oms-declara-fim-da-emergencia-saude-publica-importancia-internacional-referente). 
>
> Este anúncio traz alivio e fim a 3 longos anos, marcados por muita incerteza, perdas e mudanças talvez incomensuráveis na sociedade e no relacionamento humano.
>
> `Este projeto visa analisar os dados da COVID 19 no Brasil e extrair insights de como a mesma se comportou no país nos últimos 3 anos.`

#### 1.2 - Sobre os Dados

> Neste projeto, irei utilizar os dados coletados e compilados pelo Centro de Ciência de Sistemas e Engenharia da universidade americana **John Hopkins** ([link](https://www.jhu.edu)). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados, etc.). O site do projeto pode ser acessado neste [link](https://systems.jhu.edu/research/public-health/ncov/) enquanto os dados, neste [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **state**: estado;
 - **country**: país; 
 - **population**: população estimada;
 - **confirmed**: número acumulado de infectados;
 - **confirmed_1d**: número diário de infectados;
 - **confirmed_moving_avg_7d**: média móvel de 7 dias do número diário de infectados;
 - **confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **deaths**: número acumulado de mortos;
 - **deaths_1d**: número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias do número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **month**: mês de referência;
 - **year**: ano de referência.

#### 1.3 - Sobre o Autor

> Olá!
>
> Me chamo Leonardo, sou estudante do último ano de Engenharia Mecânica que tem como objetivo profissional migrar para a área de dados. Durante os anos de faculdade, trabalhei durante 2 anos com pesquisa na área aeroespacial (lançadores de satélite) no Instituto de Aeronáutica e Espaço, em São José dos Campos (IAE-DCTA). Lá tive meu primeiro contato com a programação e com dados (maioria proveniente de sensores acoplados aos foguetes) e, unindo essa experiencia com meu interesse pelas aulas de estatística durante a graduação, acabei conhecendo a área de ciencia de dados e me apaixonando.

#### 1.4 - Setup

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
from typing import Iterator
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 2. Extração dos Dados

## 2.1 Realizando a Extração

<font color = red><b>Atenção:</b></font> Esta seção não precisa ser executada. Para rodar o notebook, realizar o carregamento do arquivo utilizando o Pandas através do arquivo `filtered-raw-data.pkl`.

> Os dados estão salvos no formato `.csv`, onde cada arquivo corresponde a um dia do ano.

* Criando um `Generator` para criar nosso `Iterator`

In [ ]:
def date_range(data_inicial: datetime, data_final: datetime) -> Iterator:

    """
    Generator.
    Essa função recebe duas datas como input (uma data inicial e outra final), criando um Iterator
    com todas as datas entre elas. 
    A granularidade é em dias.

    Exemplo:\n\n
    >> Input\n
    data_inicial = 01/01/2021\n
    data_final = 05/01/2021\n
    date_range(data_inicial, data_final)\n
    \n
    >> Output\n
    Iterator contendo os valores: 01/01/2021, 02/01/2021, 03/01/2021, 04/01/2021 e 05/01/2021.
    """
    # Calculando a quantidade de dias
    days_range = (data_final - data_inicial).days

    # Criando o Iterator
    for lag in range(days_range):
        yield data_inicial + timedelta(lag)

* Setando a data inicial e final

In [ ]:
data_inicial = datetime(2021, 1, 1)
data_final = datetime(2023, 1, 1)

* Extraindo os dados

In [ ]:
# Lista para salvar as datas que não foram carregadas
failed = list()

# Objeto para criar o data frame Pandas
df = pd.DataFrame()

# Iterando sob todas as datas entre o intervalo que queremos considerar
for date in date_range(data_inicial = data_inicial, data_final = data_final):
    # Convertendo a data para string
    date_str = date.strftime('%m-%d-%Y')

    # Acessando o .csv através da url
    source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

    
    try: # Tenta acessar o arquivo referente à data da iteração atual

        case = pd.read_csv(source_url, sep=',')

    except: # Caso dê algum erro (possivelmente o arquivo não exista)

        failed.append(date_str)
        continue

    else: # Caso consiga ler com sucesso o arquivo, adicionamos ao data frame

        # Removendo as colunas que não iremos utilizar
        case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
        # Filtrando apenas os dados do Brasil
        case = case.query('Country_Region == "Brazil"').reset_index(drop= True)
        # Convertendo a coluna de Data
        case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))
        # Inserindo os dados dessa iteração ao data frame
        df = pd.concat([df, case], axis= 0, ignore_index = True)

* Uma vez que o dataframe extraído é pequeno (e o processo de extração é relativamente demorado), irei exporta-lo em `.csv` caso haja necessidade de resetar o kernel.

In [ ]:
df.to_pickle('./filtered-raw-data.pkl', protocol= 5)

## 2.2 Checando os Resultados da Extração

* Carregando os dados

In [3]:
df = pd.read_pickle('./filtered-raw-data.pkl')

* Checando o resultado da extração dos dados (shape, dtypes e nulos)

In [4]:
df.head(5)

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


In [5]:
# A quantidade de linhas faz sentido (27 estados x 730 dias)
df.shape

(19710, 6)

In [6]:
# Não temos dados nulos e os data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19710 entries, 0 to 19709
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  19710 non-null  object        
 1   Country_Region  19710 non-null  object        
 2   Confirmed       19710 non-null  int64         
 3   Deaths          19710 non-null  int64         
 4   Incident_Rate   19710 non-null  float64       
 5   Date            19710 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 924.0+ KB


# 3. Data Wrangling

* Ajustando o nome das colunas

In [7]:
# Mudando o nome das colunas Province_State e Country_Region
df = df.rename(
    columns=
    {
        'Province_State' : 'state',
        'Country_Region' : 'country'
    }
)

In [8]:
# Passando o nome de todas as colunas para minúsculo
df.columns = [col.lower() for col in df.columns]

* Ajustando o nome dos estados

(Importante para o Looker Studio reconhece-los como uma localização geográfica)

In [10]:
# Criando um mapper com os nomes
mapper = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Maranhao' : 'Maranhão',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

# Aplicando o mapper na variável 'state'
df['state'] = df['state'].apply(lambda state: mapper.get(state) if state in mapper.keys() else state)

# Mostrando o resultado da operação
df['state'].unique()

array(['Acre', 'Alagoas', 'Amapá', 'Amazonas', 'Bahia', 'Ceará',
       'Distrito Federal', 'Espírito Santo', 'Goiás', 'Maranhão',
       'Mato Grosso', 'Mato Grosso do Sul', 'Minas Gerais', 'Pará',
       'Paraíba', 'Paraná', 'Pernambuco', 'Piauí', 'Rio Grande do Norte',
       'Rio Grande do Sul', 'Rio de Janeiro', 'Rondônia', 'Roraima',
       'Santa Catarina', 'São Paulo', 'Sergipe', 'Tocantins'],
      dtype=object)

* Criando chaves temporais: Ano-Mês

In [14]:
df['month'] = df['date'].map(lambda date: date.strftime('%Y-%m'))

* Criando chaves temporais: Ano

In [17]:
df['year'] = df['date'].map(lambda date: date.strftime('%Y'))

* Estimando a população de cada Estado.

Sabemos que a incidencia (`incident rate`) é uma estatística que mede a ocorrência de novos casos baseado na população total que está exposta ao risco. Como sabemos que esta taxa, neste estudo, está baseada em **100.000** habitantes, podemos encontrar esses valores da seguinte maneira:

$$incident\hspace{1mm}rate = \dfrac{100.000 \times confirmed}{population}$$

Analogamente, conseguimos estimar a população total da seguinte maneira:


$$population = \dfrac{100.000 \times confirmed}{incident\hspace{1mm}rate}$$

In [20]:
df['population'] = round(100000 * (df['confirmed'] / df['incident_rate']))

In [21]:
# Removendo a coluna incident_rate, já que ela nos dá pouca informação
df = df.drop(['incident_rate'], axis= 1)

* Trabalhando com número de casos confirmados:

1. Casos diários

2. Média Móvel (7 dias)

3. Estabilidade (14 dias)

4. Definindo a tendencia da doença (crescente, decrescente ou estável) com base na estabilidade

In [27]:
def tendencia(rate: float) -> str:
    
    """
    Função criada para determinar a tendencia de desenvolvimento da doença.\n
    Caso a taxa seja maior que 1.15, a contaminação é considerada crescente.\n
    Caso a taxa seja menor que 0.75, a contaminação é considerada decrescente.\n
    Para outras situações, considera-se a mesma como estável.
    \n
    >> Input\n
    rate (float): Razão entre a média móvel atual com a média móvel anterior.
    \n
    >> Output\n
    status(str): Tendencia de desenvolvimento da doença.
    """
    if np.isnan(rate):
        return np.NaN
    
    if rate <= 0.75:
        status = 'downward'
    elif rate <= 1.15:
        status = 'upward'
    else:
        status = 'stable'
    
    return status

In [60]:
df_cases_wrangled = pd.DataFrame()

for state in df['state'].unique():
    
    df_queried = (
        df
        .query('state == @state') # Filtra pelo estado
        .reset_index(drop= True) # Arruma a numeração das linhas
        .sort_values(by= 'date') # Ordena pela data
        .copy() # Cria uma cópia, evitando que o original seja alterado
    )
    
    # Calculando o número de casos/mortes por dia
    df_queried['confirmed_1d'] = df_queried['confirmed'].diff(periods= 1)
    df_queried['deaths_1d'] = df_queried['deaths'].diff(periods = 1)
    
    # Calculando a média móvel de casos/mortes dos últimos 7 dias
    df_queried['confirmed_moving_avg_7d'] = np.ceil(df_queried['confirmed_1d'].rolling(window= 7).mean())
    df_queried['deaths_moving_avg_7d'] = np.ceil(df_queried['deaths_1d'].rolling(window= 7).mean())
    
    # Calculando a taxa de desenvolvimento da doença (comparando a média móvel atual com a anterior)
    df_queried['confirmed_moving_avg_7d_rate_14'] = df_queried['confirmed_moving_avg_7d']/df_queried['confirmed_moving_avg_7d'].shift(periods= 14)
    df_queried['deaths_moving_avg_7d_rate_14'] = df_queried['deaths_moving_avg_7d']/df_queried['deaths_moving_avg_7d'].shift(periods= 14)
    
    # Adicionando a coluna com o indicador do desenvolvimento da doença
    df_queried['confirmed_trend'] = df_queried['confirmed_moving_avg_7d_rate_14'].apply(tendencia)
    df_queried['deaths_trend'] = df_queried['deaths_moving_avg_7d_rate_14'].apply(tendencia)
    
    df_cases_wrangled = pd.concat([df_cases_wrangled, df_queried], axis= 0, ignore_index= True)